# Extraction des données complètes pour le Rhones

In [1]:
import requests
import pandas as pd
from concurrent.futures import ThreadPoolExecutor
import time

# URL de base de l'API
base_url = "https://data.ademe.fr/data-fair/api/v1/datasets/dpe03existant/lines"

In [2]:
# Chargement du ficher des adresses du rhones.
df69 = pd.read_csv("data/adresses-69.csv", sep=";", low_memory=False)

In [3]:
# Extraction des codes postaux.
code_postaux = df69["code_postal"].unique()

# Methode de requête de l'API

In [4]:
def query_api(params: dict, next_url: str | None = None, retries: int = 3, backoff: int = 2) -> list[dict]:
    """Récupère toutes les pages d'une requête API en boucle avec gestion des erreurs."""

    all_results = []

    recuperated = 0

    # Recuperation du code postal des paramètres.
    postal_code = params["qs"].split(":")[-1]

    while True:
        for attempt in range(retries):
            try:
                if next_url:
                    response = requests.get(url=next_url)
                else:
                    response = requests.get(url=base_url, params=params)

                if response.status_code == 200:

                    content = response.json()

                    # Monitoring
                    recuperated += content["total"] if content["total"] < params["size"] else params["size"]

                    

                    if content["total"] == 0:
                        print("No data found from the query.", flush=True)
                        return []
                
                    
                    all_results.extend(content["results"])
                    print(f"{postal_code}: {recuperated} éléments sur {content["total"]} ({int(recuperated*100/content["total"])}%)", flush=True)

                    next_url = content.get("next")
                    break  # sortie du retry loop si succès

                # Gestion d'erreur si requête invalide.
                else:
                    print(f"{postal_code}: Erreur de connexion avec l'API!, tentative {attempt+1}/{retries}", flush=True)
                    time.sleep(backoff * (attempt + 1))  # backoff progressif

            # Gestion d'erreur si problème de connexion.
            except requests.RequestException as e:
                print(f"{postal_code}: Erreur réseau: {e}, tentative {attempt+1}/{retries}", flush=True)
                time.sleep(backoff * (attempt + 1))  # backoff progressif

        # else de boucle for (si trop d'échecs.)
        else:
            print("{postal_code}: Échec permanent, abandon de cette requête.")
            return all_results # Retourner les informations récupérés jusqu'à présent.

        if not next_url:
            break

    return all_results

## Extraction normale

In [ ]:
df_list = list()

for i, code in enumerate(code_postaux):

    params = {
            "size": 10000,   
            "qs": f"code_postal_ban:{code}"
        }
    df_list.extend(query_api(params=params))

# Concaténation de la liste de dictionnaire en dataframe.
df = pd.DataFrame(df_list)

df.to_csv("existants_69.csv")


## Multitreading code

Le code finit en erreur, car trop de requêtes simultannées sont demandées à l'API (l'API coupe la connection).

In [ ]:
def fetch_for_code(code):
    """Exécute query_api pour un code postal donné."""
    params = {
        "size": 10000,
        "qs": f"code_postal_ban:{code}"
    }
    return query_api(params=params, retries=5)


# Liste des DataFrames pour tous les codes postaux
with ThreadPoolExecutor(max_workers=5) as executor: 
    all_lists = list(executor.map(fetch_for_code, code_postaux))

list_to_df = []
for sublist in all_lists:
    list_to_df.extend(sublist)

# Concaténation finale
df_final = pd.DataFrame(list_to_df)

In [13]:
# Vérification de l'extraction.
df = pd.read_csv("existants_69.csv", index_col=0)

C:\Users\axel-\AppData\Local\Temp\ipykernel_83784\61002706.py:2: DtypeWarning: Columns (41) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("existants_69.csv", index_col=0)


# Batiment neuf

Utiliser l'URL suivante: https://data.ademe.fr/data-fair/api/v1/datasets/dpe02neuf/lines

In [ ]:
# Récupération des logements neufs.
new_habitation_list = list()

base_url = "https://data.ademe.fr/data-fair/api/v1/datasets/dpe02neuf/lines"

for i, code in enumerate(code_postaux):

    params = {
            "size": 10000,   
            "qs": f"code_postal_ban:{code}"
        }

    new_habitation_list.extend(query_api(params=params))

# Concaténation de la liste de dictionnaire en dataframe.
df_news = pd.DataFrame(df_list)

# Ecriture du fichier en csv.
df_news.to_csv("neufs_69.csv")
